01 Define dataset index

In [442]:
import pandas as pd
from typing import Union, Tuple, List, Dict
import logging
import re
import numpy as np
import string
from IPython.display import display, HTML
from datetime import datetime, date

desired_dataset_index = input("Insert number for dataset or Database.table for table")

print(desired_dataset_index)
  
try:
    desired_dataset_index.isdigit()
    desired_dataset_index = int(desired_dataset_index)
    print(f"Dataset#: {desired_dataset_index} ")
    DB_or_dataset = "Dataset"
    datasets_file_path = 'Fortydatasets.xlsx'  
    analysed_columns_file_path = 'AnalysedColumns.xlsx'     
except ValueError:
    print(f"Database.table: {desired_dataset_index} ")
    DB_or_dataset = "DB"
    datasets_file_path = 'all_selected_databases_info.xlsx'
    analysed_columns_file_path = 'AnalysedColumnsDB.xlsx'

from datetime import datetime
print(f"Last run on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# Load the sheet to obtain the details of all datasets
datasets_df = pd.read_excel(datasets_file_path)
datasets_df.head()

analysed_columns_df = pd.read_excel(analysed_columns_file_path)


UW_std.person
Database.table: UW_std.person 
Last run on: 2024-09-24 16:03:20


02 Read Analysed Columns, define dataset index and provide CHANGED dataset file

In [443]:
analysed_columns_df = pd.read_excel(analysed_columns_file_path)

# Display the first few rows to verify the loaded data
print(f"desired_dataset_index: {desired_dataset_index} Last run on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
analysed_columns_df.head()

desired_dataset_index: UW_std.person Last run on: 2024-09-24 16:03:20


,index,name,area,Original Column,ID,Column,Description,CleanedColumn,ColumnKeyword,ColumnFormat,DescriptionKeyword,DescriptionFormat,FinalFormat,SourceKeyword
0,AustralianFootball.players,players,Sport,1. pid (int),1,pid,integer,pid,id,IDcolumn,integer,numerical,IDcolumn,id
1,AustralianFootball.players,players,Sport,2. dob (datetime),2,dob,datetime,dob,date of birth,date,datetime,datetime,datetime,datetime
2,AustralianFootball.players,players,Sport,3. first_name (varchar(255)),3,first_name,varchar 255,first name,name,name,varchar,string,name,name
3,AustralianFootball.players,players,Sport,4. last_name (varchar(255)),4,last_name,varchar 255,last name,name,name,varchar,string,name,name
4,AustralianFootball.players,players,Sport,5. height (int),5,height,integer,height,height,numerical>=0,integer,numerical,numerical>=0,height


025 Get Database Table Content

In [444]:
import pymysql
import pandas as pd

def get_db_table_content(index: str) -> pd.DataFrame:
    """
    Connect to the database and retrieve the content of the specified table.
    
    Parameters:
    - index (str): The index in the format 'database.table'
    
    Returns:
    - pd.DataFrame: The content of the table as a DataFrame
    """
    host = 'db.relational-data.org'
    port = 3306
    user = 'guest'
    password = 'relational'

    db_name, table_name = index.split('.')
    
    try:
        connection = pymysql.connect(host=host, port=port, user=user, password=password, database=db_name)
        with connection.cursor() as cursor:
            cursor.execute(f"SELECT * FROM `{table_name}`")
            data = cursor.fetchall()
            columns = [column[0] for column in cursor.description]
        
        df = pd.DataFrame(data, columns=columns)
        print(f"Successfully retrieved data from {index}")
        return df
    except Exception as e:
        print(f"Error retrieving data from {index}: {str(e)}")
        return pd.DataFrame()
    finally:
        if 'connection' in locals():
            connection.close()

03 Get dataset file URL


In [445]:
import pandas as pd
from typing import Union, Tuple, List
import logging

logging.basicConfig(level=logging.INFO)

def get_dataset_file_url(excel_file_path: str, dataset_index: Union[int, str], DB_or_dataset: str) -> Union[Tuple[str, str, List[str]], Exception]:
    """
    Load dataset details from an Excel file and return the dataset file URL and name for a specific dataset index.

    Parameters:
    - excel_file_path (str): The path to the Excel file containing dataset details.
    - dataset_index (int or str): The index of the dataset or the database table name.
    - DB_or_dataset (str): Indicates whether we're dealing with a dataset or a database table.

    Returns:
    - Tuple or Exception: A tuple containing the dataset file URL, name, and primary key columns (for DB only), or an exception if something goes wrong.
    """
    try:
        datasets_df = pd.read_excel(excel_file_path)
        
        if 'index' not in datasets_df.columns or 'name' not in datasets_df.columns:
            return "Required columns 'index' and 'name' do not exist in the DataFrame"

        if DB_or_dataset == "Dataset":
            # Dataset case
            dataset_index = int(dataset_index)
            dataset_details = datasets_df.loc[datasets_df['index'] == dataset_index]
            if dataset_details.empty:
                return f"No dataset found with index {dataset_index}"

            dataset_name = dataset_details['name'].values[0]
            dataset_file_url = dataset_details['dataset_file_url'].values[0]
            
            # Datasets don't have primary keys
            primary_key_columns = []

        elif DB_or_dataset == "DB":
            # Database table case
            dataset_details = datasets_df.loc[datasets_df['index'] == dataset_index]
            if dataset_details.empty:
                return f"No dataset found with index {dataset_index}"

            dataset_name = dataset_details['name'].values[0]
            dataset_file_url = dataset_index  # For database tables, the index is the URL
            
            # Extract primary key columns for DB tables
            if 'primary_key' in datasets_df.columns:
                primary_key = dataset_details['primary_key'].values[0]
                primary_key_columns = [col.strip() for col in str(primary_key).split(',')] if pd.notna(primary_key) else []
                print(f"Primary key columns extracted: {primary_key_columns}")  # Debug print
            else:
                primary_key_columns = []
                print("No primary key column found in the DataFrame")
        
        else:
            return f"Invalid DB_or_dataset value: {DB_or_dataset}"

        return (dataset_file_url, dataset_name, primary_key_columns)

    except Exception as e:
        logging.error(f"An error occurred: {e}")
        return e

# Example usage
if __name__ == "__main__":
    excel_file_path = datasets_file_path
    dataset_details = get_dataset_file_url(excel_file_path, desired_dataset_index, DB_or_dataset)
    
    if isinstance(dataset_details, tuple):
        dataset_file_url, dataset_name, primary_key_columns = dataset_details
        print(f"The dataset file URL or database index for dataset {dataset_name} (index {desired_dataset_index}) is: {dataset_file_url}")
        if DB_or_dataset == "DB":
            print(f"Primary key columns: {primary_key_columns}")
        print(DB_or_dataset)
    else:
        print(f"An error occurred or the dataset was not found: {dataset_details}")

Primary key columns extracted: ['p_id']
The dataset file URL or database index for dataset person (index UW_std.person) is: UW_std.person
Primary key columns: ['p_id']
DB


04 Load dataset

In [446]:
import pandas as pd
import requests
import io
from io import BytesIO
import zipfile
import tarfile
import gzip
import logging
import csv
from typing import Optional, Union
import os
import tempfile
import shutil
import sys

logging.basicConfig(level=logging.INFO)

def is_header_for_csv(line, delimiter=' '):
    """
    Determine if a line is likely a header by checking if there are no numeric values.
    If at least one numeric value is found, the line is considered not a header (i.e., a data line).
    """
    # Regex to match quoted strings or non-whitespace sequences
    pattern = re.compile(r'\".*?\"|\S+')

    elements = pattern.findall(line.replace(delimiter, ' '))  # Replace delimiter with space for easier parsing

    # Check if any element is a number
    is_numeric_present = any(element.replace('.', '', 1).lstrip('-').isdigit() for element in elements)

    # Check for replicated elements
    unique_elements = set(elements)
    replicated_elements = len(elements) - len(unique_elements)

    # If no numeric value is found and there are replicated elements, consider this a header line
    is_header = not is_numeric_present and replicated_elements == 0

    return is_header


def load_csv(file_content: Union[str, bytes], header: Optional[int], na_values: Optional[Union[str, list]]) -> pd.DataFrame:
    # Adjust to decode bytes if necessary
    if isinstance(file_content, bytes):
        try:
            file_content_decoded = file_content.decode('utf-8')
        except UnicodeDecodeError:
            file_content_decoded = file_content.decode('ISO-8859-1')
    else:
        file_content_decoded = file_content
    
    # Preprocess to replace multiple tabs with a single tab
    #file_content_processed = re.sub('\t+', '\t', file_content_decoded)
    file_content_processed = re.sub('[ \t]+', ' ', file_content_decoded)

    file_stream = io.StringIO(file_content_processed)
        
    try:
        dialect = csv.Sniffer().sniff(file_stream.readline())
        delimiter = dialect.delimiter
        file_stream.seek(0)
    except csv.Error:
        delimiter = ','
        logging.info("Falling back to default delimiter ',' due to detection failure.")
  
    # Use the heuristic to decide if the first line is likely a header
    first_line = file_stream.readline()
    # Print the first row for inspection
    print("First row for inspection:", first_line)
    is_header_row = is_header_for_csv(first_line, delimiter)
    print('Is first row header ?',is_header_row )
    file_stream.seek(0)  # Reset to start of file after reading the first line
    
    header_decision = 0 if is_header_for_csv(first_line, delimiter) else None
    #header_decision = None if is_data_row else 0
    # Regex pattern to match quoted strings or non-whitespace sequences
    pattern = re.compile(r'\".*?\"|\S+')

    df = pd.read_csv(file_stream, header=header_decision, delimiter=delimiter, na_values=na_values, keep_default_na=False)
   
  # Check if the DataFrame needs re-parsing with regex pattern
    if len(df.columns) < 2:
        file_stream.seek(0)
        lines = file_stream.readlines()
        parsed_data = [pattern.findall(line) for line in lines]

        if header is None and header_decision == 0:
            header_row = parsed_data.pop(0)  # Use the first row as header
        else:
            header_row = None  # Let pandas create default headers or use provided header index

        df = pd.DataFrame(parsed_data, columns=header_row)
        df = df.apply(pd.to_numeric, errors='ignore')  # Attempt to correct data types
    return df

def is_header_for_excel(first_row: pd.DataFrame) -> bool:
    """
    Determines if the first row of a DataFrame is likely to be a header by checking the entire row's content.

    Args:
        first_row (pd.DataFrame): DataFrame containing at least one row of data to inspect.

    Returns:
        bool: True if the first row is likely a header (i.e., contains mostly non-numeric data), False if it's likely data.
    """
    # Check if the majority of the items in the first row are non-numeric
    non_numeric_count = first_row.iloc[0].apply(lambda x: not isinstance(x, (int, float))).sum()
    
    # Determine if the first row is likely a header based on the proportion of non-numeric items
    is_header = non_numeric_count > len(first_row.columns) / 2
    return is_header


def load_excel(file_content: Union[str, bytes, io.BytesIO], na_values: Optional[Union[str, list]] = None, skip_rows: Union[int, list] = 0, parse_dates: bool = False) -> pd.DataFrame:
    # Ajuste para o file_content ser um objeto BytesIO, se for bytes
    if isinstance(file_content, bytes):
        file_content = BytesIO(file_content)  # Envolve bytes em BytesIO
    try:
        # Load the first row to check if it's likely to be a header
        first_row = pd.read_excel(file_content, nrows=1, header=None)
        
        # Print the first row for inspection
        print("First row for inspection:", first_row.iloc[0].values)

        # Execute the heuristic
        likely_header = is_header_for_excel(first_row)

        # Decide on using the first row as header based on heuristic
        #header_decision = None if likely_header else 0  
        header_decision = 0 if likely_header else None

        # Reset file_content to read from the beginning if it's a stream
        if isinstance(file_content, io.BytesIO):
            print("Resetting stream position")  # Diagnostic print
            file_content.seek(0)

        # Load the full Excel file with determined header option
        df = pd.read_excel(file_content, na_values=na_values, skiprows=skip_rows, header=header_decision, parse_dates=parse_dates)
        return df
    except ValueError as e:
        logging.error(f"Excel loading error: {e}")
        return pd.DataFrame()
        
def download_and_extract(url: str) -> str:
    """
    Downloads an archive from the given URL and extracts it into a temporary directory.
    Handles .zip, .tar.gz, and .gz files. Returns the path to the directory or file.
    """
    temp_dir = tempfile.mkdtemp()
    response = requests.get(url, stream=True)
    file_name = os.path.basename(url)
    temp_file_path = os.path.join(temp_dir, file_name)
    
    with open(temp_file_path, 'wb') as file:
        shutil.copyfileobj(response.raw, file)
    
    if file_name.endswith('.zip'):
        with zipfile.ZipFile(temp_file_path, 'r') as archive:
            archive.extractall(temp_dir)
    elif file_name.endswith(('.tar.gz', '.tgz')):
        with tarfile.open(temp_file_path, 'r:gz') as archive:
            archive.extractall(temp_dir)
    elif file_name.endswith('.gz'):
        # Handle single .gz files by extracting to the same directory
        extracted_file_path = temp_file_path[:-3]  # Remove .gz extension
        with gzip.open(temp_file_path, 'rb') as f_in, open(extracted_file_path, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)
        os.remove(temp_file_path)  # Remove the original .gz file
        return extracted_file_path  # Direct path to the extracted file for .gz
    
    os.remove(temp_file_path)  # Clean up archive file after extraction
    return temp_dir  # Path to directory with extracted files

def select_file_from_extracted(directory: str) -> Optional[str]:
    """
    Recursively lists all files in the extracted directory and prompts the user to select one.
    """
    files_list = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            files_list.append(os.path.join(root, file))

    if not files_list:
        logging.error("No files found in the directory.")
        return None

    for index, file in enumerate(files_list, start=1):
        relative_path = os.path.relpath(file, directory)  # Show relative path for clarity
        print(f"{index}: {relative_path}", flush=True)

    try:
        file_index = int(input("Enter the number of the file you want to load: ")) - 1
        if 0 <= file_index < len(files_list):
            selected_relative_path = os.path.relpath(files_list[file_index], directory)  # Get relative path
            print(f"Selected File: {selected_relative_path}")  # Print relative path
            return files_list[file_index]  # Return full path for further processing
        else:
            logging.error("Invalid selection.")
    except ValueError:
        logging.error("Please enter a valid number.")
    
    return None

def fetch_file_content(url: str) -> str:
    response = requests.get(url, stream=True)
    try:
        content = response.content.decode('utf-8')
    except UnicodeDecodeError:
        content = response.content.decode('ISO-8859-1')
    return content

   
def load_dataset(file_path_or_url: str, DB_or_dataset: str, na_values: Optional[Union[str, list]] = None, skip_rows: Union[int, list] = 0, parse_dates: bool = False) -> Union[pd.DataFrame, str]:

    if DB_or_dataset == "DB":
        try:
            df = get_db_table_content(file_path_or_url)
            if len(primary_key_columns) > 1:
                composite_pk_name = '|'.join(primary_key_columns)
                df[composite_pk_name] = df[primary_key_columns].astype(str).agg('|'.join, axis=1)
                # Move the composite key column to the front
                cols = df.columns.tolist()
                cols = [composite_pk_name] + [col for col in cols if col != composite_pk_name]
                df = df[cols]
            return df
        except Exception as e:
            logging.error(f"Database load error: {e}")
            return None
        
    # Handle files or URLs       
    elif file_path_or_url.startswith(('http://', 'https://')):
        if file_path_or_url.endswith(('.csv', '.txt', '.data')):
            response = requests.get(file_path_or_url)
            file_content = response.content
            return load_csv(file_content, header=None, na_values=na_values)
        elif file_path_or_url.endswith(('.xlsx', '.xls')):
            response = requests.get(file_path_or_url)
            return load_excel(response.content, na_values=na_values, skip_rows=skip_rows, parse_dates=parse_dates)
    else:
        # Handle local file path
        if os.path.exists(file_path_or_url):
            if file_path_or_url.endswith(('.csv', '.txt', '.data')):
                with open(file_path_or_url, 'r', encoding='utf-8') as f:
                    file_content = f.read()
                return load_csv(file_content, header=None, na_values=na_values)
            elif file_path_or_url.endswith(('.xlsx', '.xls')):
                with open(file_path_or_url, 'rb') as f:
                    file_content = f.read()
                return load_excel(file_content, na_values=na_values, skip_rows=skip_rows, parse_dates=parse_dates)
        else:
            logging.error(f"File does not exist: {file_path_or_url}")
            return None
    # Handling archives
    extracted_path = download_and_extract(file_path_or_url)
    if os.path.isdir(extracted_path):
        selected_file = select_file_from_extracted(extracted_path)
        if selected_file and selected_file.endswith(('.csv', '.txt', '.data')):
            with open(selected_file, 'rb') as f:
                file_content = f.read()
            return load_csv(file_content, header=None, na_values=na_values)
        elif selected_file and selected_file.endswith(('.xlsx', '.xls')):
            with open(selected_file, 'rb') as f:
                file_content = f.read()
            return load_excel(file_content, na_values=na_values, skip_rows=skip_rows, parse_dates=parse_dates)
    elif os.path.isfile(extracted_path):
        if extracted_path.endswith(('.csv', '.txt', '.data')):
            with open(extracted_path, 'rb') as f:
                file_content = f.read()
            return load_csv(file_content, header=None, na_values=na_values)
        elif extracted_path.endswith(('.xlsx', '.xls')):
            with open(extracted_path, 'rb') as f:
                file_content = f.read()
            return load_excel(file_content, na_values=na_values, skip_rows=skip_rows, parse_dates=parse_dates)
        else:
            # Attempt to load the file without assuming an extension, particularly useful for .gz extracted files
            with open(extracted_path, 'rb') as f:
                file_content = f.read()
            try:
                # Attempt to load as CSV first; this part assumes CSV if no extension is found
                return load_csv(file_content, header=None, na_values=na_values)
            except Exception as e:
                logging.error(f"Failed to automatically determine file type for {extracted_path}: {e}")
                return None
    return None

# Example usage
if __name__ == "__main__":
    excel_file_path = datasets_file_path
    dataset_details = get_dataset_file_url(excel_file_path, desired_dataset_index, DB_or_dataset)

    if isinstance(dataset_details, tuple):
        dataset_file_url, dataset_name, primary_key_columns = dataset_details
        print(f"The dataset file URL or database index for dataset {dataset_name} (index {desired_dataset_index}) is: {dataset_file_url}")
        print(f"Primary key columns: {primary_key_columns}")
        print(DB_or_dataset)
        dataset_df = load_dataset(dataset_file_url, DB_or_dataset, primary_key_columns)
        if dataset_df is not None:
            print("Dataset loaded successfully.")
            if len(primary_key_columns) > 1:
                composite_pk_name = '|'.join(primary_key_columns)
                print(f"\nComposite Primary Key ({composite_pk_name}):")
                print(dataset_df[[composite_pk_name] + [col for col in dataset_df.columns if col != composite_pk_name]].head())
        else:
            print("Failed to load dataset.")
    else:
        print(f"An error occurred or the dataset was not found: {dataset_details}")
       
from datetime import datetime
print(f"Last run on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
dataset_df      


Primary key columns extracted: ['p_id']
The dataset file URL or database index for dataset person (index UW_std.person) is: UW_std.person
Primary key columns: ['p_id']
DB
Successfully retrieved data from UW_std.person
Dataset loaded successfully.
Last run on: 2024-09-24 16:03:22


,p_id,professor,student,hasPosition,inPhase,yearsInProgram
0,3,0,1,0,0,0
1,4,0,1,0,0,0
2,5,1,0,Faculty,0,0
3,6,0,1,0,Post_Quals,Year_2
4,7,1,0,Faculty_adj,0,0
...,...,...,...,...,...,...
273,428,0,1,0,0,0
274,429,0,1,0,Post_Quals,Year_5
275,431,0,1,0,Pre_Quals,Year_2
276,432,0,1,0,Post_Quals,Year_5


05 Assign Column names

In [447]:
def assign_column_names(analysed_columns_df: pd.DataFrame, dataset_index: Union[int, str], dataset_df: pd.DataFrame, dataset_name: str = "Unknown") -> Union[pd.DataFrame, Exception]:
    """
    Assign column names to a DataFrame based on a given dataset index from an "AnalysedColumns" DataFrame.

    Parameters:
    - analysed_columns_df (pd.DataFrame): The DataFrame containing analysed columns information.
    - dataset_index (Union[int, str]): The index of the dataset for which to get the column names.
    - dataset_df (pd.DataFrame): The DataFrame to which the column names will be assigned.
    - dataset_name (str): The name of the dataset. Default is "Unknown".

    Returns:
    - pd.DataFrame or Exception: The DataFrame with assigned column names or an exception if something goes wrong.
    """
    print(f"Assigning column names for dataset index: {dataset_index}")
    print(f"Type of dataset_index: {type(dataset_index)}")
    print(f"Shape of dataset_df: {dataset_df.shape}")
    print(f"Current columns of dataset_df: {dataset_df.columns.tolist()}")
    
    try:
        # Check if 'index' and 'Column' columns exist in the DataFrame
        if 'index' not in analysed_columns_df.columns or 'Column' not in analysed_columns_df.columns:
            return f"Required columns 'index' or 'Column' do not exist in the DataFrame"

        # Extract the column names for the given dataset index
        column_names = analysed_columns_df.loc[analysed_columns_df['index'] == dataset_index, 'Column'].tolist()
        print(f"Extracted column names: {column_names}")
        
        if not column_names:
            print(f"No column names found. Unique values in 'index' column: {analysed_columns_df['index'].unique()}")
            return f"No column names found for dataset index {dataset_index}"

        # Check if there's a mismatch in the number of columns
        if len(column_names) != len(dataset_df.columns):
            print(f"Warning: Mismatch in number of columns. DataFrame has {len(dataset_df.columns)} columns, but {len(column_names)} names were provided.")
            print(f"Current DataFrame columns: {dataset_df.columns.tolist()}")
            print(f"Provided column names: {column_names}")

            # If there's an extra column (likely the composite primary key), keep it
            if len(dataset_df.columns) == len(column_names) + 1 and '|' in dataset_df.columns[0]:
                composite_pk_name = dataset_df.columns[0]
                new_column_names = [composite_pk_name] + column_names
                dataset_df.columns = new_column_names
                print(f"Kept composite primary key column '{composite_pk_name}' and assigned provided names to other columns.")
                
                # Remove the individual columns that make up the composite key
                individual_pk_columns = composite_pk_name.split('|')
                dataset_df = dataset_df.drop(columns=individual_pk_columns)
                print(f"Removed individual primary key columns: {individual_pk_columns}")
            else:
                return f"Cannot assign column names due to mismatch. Please check the column definitions."
        else:
            # Assign the extracted column names to the target DataFrame
            dataset_df.columns = column_names

        logging.info(f"Successfully assigned column names to the dataset '{dataset_name}' for index {dataset_index}")
        print(f"Final columns of dataset_df: {dataset_df.columns.tolist()}")
        return dataset_df
    except Exception as e:
        logging.error(f"An error occurred while assigning column names to the dataset '{dataset_name}': {e}")
        print(f"Error details: {str(e)}")
        return e
    
# Assume analysed_columns_df contains the analysed columns information
# Assume target_df is the DataFrame loaded previously
# Assume dataset_name contains the name of the dataset

# Call the function to assign the column names
dataset_df = assign_column_names(analysed_columns_df, desired_dataset_index, dataset_df, dataset_name)

# Display the first few rows to verify the column names have been correctly assigned


from datetime import datetime
print(f"Last run on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

dataset_df


INFO:root:Successfully assigned column names to the dataset 'person' for index UW_std.person


Assigning column names for dataset index: UW_std.person
Type of dataset_index: <class 'str'>
Shape of dataset_df: (278, 6)
Current columns of dataset_df: ['p_id', 'professor', 'student', 'hasPosition', 'inPhase', 'yearsInProgram']
Extracted column names: ['p_id', 'professor', 'student', 'hasPosition', 'inPhase', 'yearsInProgram']
Final columns of dataset_df: ['p_id', 'professor', 'student', 'hasPosition', 'inPhase', 'yearsInProgram']
Last run on: 2024-09-24 16:03:22


,p_id,professor,student,hasPosition,inPhase,yearsInProgram
0,3,0,1,0,0,0
1,4,0,1,0,0,0
2,5,1,0,Faculty,0,0
3,6,0,1,0,Post_Quals,Year_2
4,7,1,0,Faculty_adj,0,0
...,...,...,...,...,...,...
273,428,0,1,0,0,0
274,429,0,1,0,Post_Quals,Year_5
275,431,0,1,0,Pre_Quals,Year_2
276,432,0,1,0,Post_Quals,Year_5


In [448]:
print(dataset_df.dtypes)

p_id               int64
professor         object
student           object
hasPosition       object
inPhase           object
yearsInProgram    object
dtype: object


In [449]:
def convert_date_column(df):
    if 'date' in df.columns:
        try:
            df['date'] = pd.to_datetime(df['date'], errors='coerce')
            print("Date column successfully converted to datetime.")
        except Exception as e:
            print(f"Error converting date column: {e}")
            print("Original date column retained.")
    else:
        print("No 'date' column found in the dataset.")
    return df

# Convert the date column
dataset_df = convert_date_column(dataset_df)

No 'date' column found in the dataset.


In [450]:
'''date_columns = ['i_rec_start_date', 'i_rec_end_date'] 
for col in date_columns:
    dataset_df[col] = pd.to_datetime(dataset_df[col])'''

"date_columns = ['i_rec_start_date', 'i_rec_end_date'] \nfor col in date_columns:\n    dataset_df[col] = pd.to_datetime(dataset_df[col])"

In [451]:
'''import pandas as pd
from dateutil import parser
from pandas.errors import OutOfBoundsDatetime

def parse_date(date_str):
    try:
        # Try pandas to_datetime first
        return pd.to_datetime(date_str)
    except OutOfBoundsDatetime:
        try:
            # If pandas fails, try dateutil parser
            return parser.parse(date_str)
        except:
            # If both fail, return the original string
            return date_str

# Apply the custom parser to the 'Independence' column
dataset_df['Independence'] = dataset_df['Independence'].apply(parse_date)'''

"import pandas as pd\nfrom dateutil import parser\nfrom pandas.errors import OutOfBoundsDatetime\n\ndef parse_date(date_str):\n    try:\n        # Try pandas to_datetime first\n        return pd.to_datetime(date_str)\n    except OutOfBoundsDatetime:\n        try:\n            # If pandas fails, try dateutil parser\n            return parser.parse(date_str)\n        except:\n            # If both fail, return the original string\n            return date_str\n\n# Apply the custom parser to the 'Independence' column\ndataset_df['Independence'] = dataset_df['Independence'].apply(parse_date)"

In [452]:
print(dataset_df.dtypes)

p_id               int64
professor         object
student           object
hasPosition       object
inPhase           object
yearsInProgram    object
dtype: object


Ydata Profiling

pip install -U ydata-profiling

In [453]:
# Standard Library Imports
from pathlib import Path

# Installed packages
import pandas as pd
from ipywidgets import widgets

# Our package
from ydata_profiling import ProfileReport
from ydata_profiling.utils.cache import cache_file

Run Profiling

In [454]:
profile = ProfileReport(dataset_df, title="Profiling Report")

In [455]:
profile.to_notebook_iframe()
#profile.to_file("tpcds.store_sales.html")


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
def check_numerical_between(df: pd.DataFrame, column: str, min_value: float, max_value: float) -> str:
    """
    Check if all values in the specified column are numerical and fall within the given range.
    Additionally, report the actual range of numeric values.
    """
    # First, handle blank/empty/null/NaN/Missing values
    result_blank = DataQualityIssues.handle_blank_empty_null_nan(df, column)

    # Only proceed with other checks if the value is not blank/empty/null/NaN/Missing
    # Filtering df based on blank checks
    blank_indices = list(result_blank["blank_indices"])
    df_filtered = df[~df.index.isin(blank_indices)]

    # Checks for non-numeric and out-of-range values
    result_non_numeric = DataQualityIssues.handle_non_numeric_values(df_filtered, column)
    result_outside_range = DataQualityIssues.handle_values_outside_range(df_filtered, column, min_value, max_value)

    error_summary_parts = []

    if result_blank['issue']:
        error_summary_parts.append(result_blank['dq_issue'] + ':\n ' + result_blank['error_message'] + '\n')

    if result_outside_range['issue']:
        error_summary_parts.append(result_outside_range['dq_issue'] + ':\n ' + result_outside_range['error_message'] + '\n')

    if result_non_numeric['issue']:
        error_summary_parts.append(result_non_numeric['dq_issue'] + ':\n ' + result_non_numeric['error_message'] + '\n')

    # Convert the column to numeric, ignoring errors to leave non-numeric as NaN
    numeric_values = pd.to_numeric(df_filtered[column], errors='coerce')

    # Filter out non-numeric values to avoid them in min/max calculations
    numeric_values_filtered = numeric_values.dropna()

    # Calculate the range (min and max) of the numeric values
    actual_min_value = numeric_values_filtered.min()
    actual_max_value = numeric_values_filtered.max()

    range_summary = f"Actual range of values: ({actual_min_value} : {actual_max_value})" if not numeric_values_filtered.empty else "No valid numeric values found."

    if not error_summary_parts:
        return f"All {len(numeric_values_filtered)} values are numerical and valid in the range [{min_value}, {max_value}].\n{range_summary}"

    error_summary = "Error(s) found: \n" + "\n".join(error_summary_parts)

    return f"{error_summary}\n{range_summary}"

# Test function
df_test = pd.DataFrame({'age': [1, 2, 131, 'a3', 5, '?', None, -1, '', "", 'NULL', '  ', 140.0, 129.0]})
result = check_numerical_between(df_test, 'age', 0, 130)
print(result)

from datetime import datetime
print(f"Last run on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


Error(s) found: 
DQI #1 (Missing Data - Completeness):
 6 Blank/Empty/Null/Missing value(s) at index(es): 
2 NULL values: [(6, 'NULL'), (10, 'NULL')]
4 Non-NULL values: [(5, '?'), (8, 'Empty'), (9, 'Empty'), (11, 'Blank')]

DQI #15 (Domain Violation - Accuracy):
 3 Value(s) outside range [0, 130] at index(es): [(2, 131), (7, -1), (12, 140.0)]

DQI #17 (Wrong Data Type - Consistency):
 1 Non-numeric value(s) at index(es): [(3, 'a3')]

Actual range of values: (-1.0 : 140.0)
Last run on: 2024-09-23 21:54:06


In [456]:
raise SystemExit

SystemExit: 

c:\Users\290099c\AppData\Local\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


CHANGED

In [228]:
# Load the CHANGED dataset from changed_dataset_local_path 
#df = pd.read_csv('your_file.csv', na_values=['', ' ', 'null', 'none', 'None', 'Nan', 'NULL'], keep_default_na=False)

#changed_df = pd.read_csv(changed_dataset_local_path, header=None, delimiter=',', na_values=['', ' ', 'null', 'none', 'None', 'Nan', 'NULL'], keep_default_na=False)
#changed_df = pd.read_csv(changed_dataset_local_path, header=None, delimiter=',', na_values=[], keep_default_na=False)

#changed_df = pd.read_csv(changed_dataset_local_path, delimiter=';', na_values=[], keep_default_na=False) #186, 222
#changed_df = pd.read_excel(changed_dataset_local_path, header=header, parse_dates=parse_dates, na_values=[], keep_default_na=False) #602 350
#changed_df.head(10)

changed_df = pd.read_csv(changed_dataset_local_path, header=None, delimiter=',', na_values=[], keep_default_na=False)
changed_df.head(10)

from datetime import datetime
print(f"Last run on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
changed_df

NameError: name 'changed_dataset_local_path' is not defined

In [ ]:
dataset_df = changed_df


# Call the function to assign the column names
dataset_df = assign_column_names(analysed_columns_df, desired_dataset_index, dataset_df, dataset_name)


from datetime import datetime
print(f"Last run on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

dataset_df.head(10)